<a href="https://colab.research.google.com/github/alaydv/isi25b/blob/main/notebooks/04datamanip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 Manipuación de los Datos:

En este ejercicio vamos a manipular una base de datos de recursos humanos

In [81]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Configuración
num_empleados = 50
departamentos = ["Ventas", "TI", "Recursos Humanos", "Finanzas", "Operaciones"]
cargos = ["Analista", "Gerente", "Asistente", "Especialista", "Director"]

# 1. Tabla: Empleados (Maestro)
empleados = []
for i in range(1, num_empleados + 1):
    id_emp = f"E{i:03d}"
    nombre = f"Empleado_{i}"
    depto = random.choice(departamentos)
    cargo = random.choice(cargos)
    salario = random.randint(800, 3500)
    fecha_ingreso = datetime(2018, 1, 1) + timedelta(days=random.randint(0, 2000))
    genero = random.choice(["M", "F"])
    empleados.append([id_emp, nombre, depto, cargo, salario, fecha_ingreso.strftime("%Y-%m-%d"), genero])

df_empleados = pd.DataFrame(empleados, columns=["ID_Empleado", "Nombre", "Departamento", "Cargo", "Salario_Mensual", "Fecha_Ingreso", "Genero"])

# 2. Tabla: Evaluaciones de Desempeño (Hechos - Anual)
# Asumimos la evaluación del último año
evaluaciones = []
for emp in empleados:
    id_emp = emp[0]
    # Score 1 to 5
    score = random.choices([1, 2, 3, 4, 5], weights=[5, 10, 40, 35, 10])[0] # Weighted towards 3 and 4
    fecha_eval = "2023-12-15"
    evaluaciones.append([id_emp, fecha_eval, score])

df_evaluaciones = pd.DataFrame(evaluaciones, columns=["ID_Empleado", "Fecha_Evaluacion", "Puntaje_Desempeno"])

# 3. Tabla: Registro de Asistencia (Hechos - Mensual Resumido)
# Datos de los últimos 3 meses para cada empleado
asistencia = []
meses = ["2024-01", "2024-02", "2024-03"]

for emp in empleados:
    id_emp = emp[0]
    for mes in meses:
        dias_ausente = random.choices([0, 1, 2, 3], weights=[70, 15, 10, 5])[0]
        horas_extra = random.randint(0, 10)
        asistencia.append([id_emp, mes, dias_ausente, horas_extra])

df_asistencia = pd.DataFrame(asistencia, columns=["ID_Empleado", "Mes", "Dias_Ausente", "Horas_Extra"])

# Save to CSV
df_empleados.to_csv("../HR_Empleados.csv", index=False)
df_evaluaciones.to_csv("../HR_Evaluaciones.csv", index=False)
df_asistencia.to_csv("../HR_Asistencia.csv", index=False)

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
print("Empleados Head:")
print(df_empleados.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nEvaluaciones Head:")
print(df_evaluaciones.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nAsistencia Head:")
print(df_asistencia.head().to_markdown(index=False, numalign="left", stralign="left"))

Empleados Head:
| ID_Empleado   | Nombre     | Departamento     | Cargo        | Salario_Mensual   | Fecha_Ingreso   | Genero   |
|:--------------|:-----------|:-----------------|:-------------|:------------------|:----------------|:---------|
| E001          | Empleado_1 | Recursos Humanos | Analista     | 1757              | 2021-02-07      | F        |
| E002          | Empleado_2 | Ventas           | Director     | 2734              | 2021-04-18      | F        |
| E003          | Empleado_3 | Finanzas         | Especialista | 2432              | 2020-02-29      | M        |
| E004          | Empleado_4 | Ventas           | Especialista | 2806              | 2023-06-03      | F        |
| E005          | Empleado_5 | Ventas           | Gerente      | 3349              | 2020-05-03      | M        |

Evaluaciones Head:
| ID_Empleado   | Fecha_Evaluacion   | Puntaje_Desempeno   |
|:--------------|:-------------------|:--------------------|
| E001          | 2023-12-15         | 3    

## 1. El Escenario de Negocio

Eres un analista de sistemas recién contratado en la empresa. El **Director de Recursos Humanos** está preocupado. Siente que el presupuesto de salarios es muy alto y no está seguro de si se está pagando justamente a las personas que más se esfuerzan.

Te ha entregado tres archivos exportados del sistema antiguo de gestión (Legacy System) y te ha dicho:
> *"Tengo estos datos sueltos y no entiendo nada. Necesito saber quiénes son mis empleados estrella y quiénes están ganando mucho dinero pero rindiendo poco. ¡Tráeme un reporte en una hora!"*

## 2. Datos

Descarga los siguientes archivos `.csv` disponibles en el aula virtual:

1.  **`HR_Empleados.csv` (Maestro de Empleados):** Contiene la información personal, cargo y salario.
    * *Llave Principal:* `ID_Empleado`
2.  **`HR_Evaluaciones.csv` (Desempeño):** Contiene el puntaje de la última evaluación anual (Escala 1 a 5).
    * *Llave Foránea:* `ID_Empleado`
3.  **`HR_Asistencia.csv` (Asistencia):** Registro mensual de los últimos 3 meses.
    * *Llave Foránea:* `ID_Empleado`

In [84]:
# Cargar datos y mostrarlos
df_empleados = pd.read_csv("../HR_Empleados.csv")
df_evaluaciones = pd.read_csv("../HR_Evaluaciones.csv")
df_asistencia = pd.read_csv("../HR_Asistencia.csv")

df_empleados.head()


,ID_Empleado,Nombre,Departamento,Cargo,Salario_Mensual,Fecha_Ingreso,Genero
0,E001,Empleado_1,Recursos Humanos,Analista,1757,2021-02-07,F
1,E002,Empleado_2,Ventas,Director,2734,2021-04-18,F
2,E003,Empleado_3,Finanzas,Especialista,2432,2020-02-29,M
3,E004,Empleado_4,Ventas,Especialista,2806,2023-06-03,F
4,E005,Empleado_5,Ventas,Gerente,3349,2020-05-03,M


In [85]:
df_evaluaciones.head()

,ID_Empleado,Fecha_Evaluacion,Puntaje_Desempeno
0,E001,2023-12-15,3
1,E002,2023-12-15,3
2,E003,2023-12-15,4
3,E004,2023-12-15,3
4,E005,2023-12-15,2


In [86]:
df_asistencia.head()

,ID_Empleado,Mes,Dias_Ausente,Horas_Extra
0,E001,2024-01,0,9
1,E001,2024-02,0,5
2,E001,2024-03,1,10
3,E002,2024-01,0,5
4,E002,2024-02,2,0


## 3. Instrucciones

### Paso 1: Unificación de Datos (Data Blending)
Los datos están fragmentados (como en la mayoría de sistemas reales). Tu primer trabajo es consolidarlos.
* Abre `HR_Empleados.csv`. Esta será tu tabla principal.
* Usa la función `BUSCARV` (VLOOKUP) en Excel o `merge` en Python para traer las siguientes columnas a tu tabla principal, usando el `ID_Empleado` como conector:
    * Trae el `Puntaje_Desempeno` desde el archivo de Evaluaciones.
    * **Reto:** Calcula el *promedio* de `Dias_Ausente` de los 3 meses (desde el archivo de Asistencia) y tráelo a la tabla principal. (Si usas Excel, quizás debas usar Tablas Dinámicas primero para resumir la asistencia por empleado).

In [87]:
# Merge con Evaluaciones

df = df_empleados.merge(df_evaluaciones[["ID_Empleado","Puntaje_Desempeno"]],
                        on="ID_Empleado", how="left")
df.head()

,ID_Empleado,Nombre,Departamento,Cargo,Salario_Mensual,Fecha_Ingreso,Genero,Puntaje_Desempeno
0,E001,Empleado_1,Recursos Humanos,Analista,1757,2021-02-07,F,3
1,E002,Empleado_2,Ventas,Director,2734,2021-04-18,F,3
2,E003,Empleado_3,Finanzas,Especialista,2432,2020-02-29,M,4
3,E004,Empleado_4,Ventas,Especialista,2806,2023-06-03,F,3
4,E005,Empleado_5,Ventas,Gerente,3349,2020-05-03,M,2


In [88]:
# Calcular promedio de días ausentes por empleado
promedio_ausencias = df_asistencia.groupby("ID_Empleado")["Dias_Ausente"].mean().reset_index()
promedio_ausencias.rename(columns={"Dias_Ausente":"Promedio_Dias_Ausente"}, inplace=True)

promedio_ausencias.head()

,ID_Empleado,Promedio_Dias_Ausente
0,E001,0.333333
1,E002,0.666667
2,E003,0.666667
3,E004,0.333333
4,E005,0.333333


In [89]:
# Merge con dia ausente
df = df.merge(promedio_ausencias, on="ID_Empleado", how="left")
df.head()

,ID_Empleado,Nombre,Departamento,Cargo,Salario_Mensual,Fecha_Ingreso,Genero,Puntaje_Desempeno,Promedio_Dias_Ausente
0,E001,Empleado_1,Recursos Humanos,Analista,1757,2021-02-07,F,3,0.333333
1,E002,Empleado_2,Ventas,Director,2734,2021-04-18,F,3,0.666667
2,E003,Empleado_3,Finanzas,Especialista,2432,2020-02-29,M,4,0.666667
3,E004,Empleado_4,Ventas,Especialista,2806,2023-06-03,F,3,0.333333
4,E005,Empleado_5,Ventas,Gerente,3349,2020-05-03,M,2,0.333333


In [90]:
# Calcular promedio de horas extra por empleado
promedio_horas = df_asistencia.groupby("ID_Empleado")["Horas_Extra"].mean().reset_index()
promedio_horas.rename(columns={"Horas_Extra":"Promedio_Horas_Extra"}, inplace=True)
df = df.merge(promedio_horas, on="ID_Empleado", how="left")
df.head()

,ID_Empleado,Nombre,Departamento,Cargo,Salario_Mensual,Fecha_Ingreso,Genero,Puntaje_Desempeno,Promedio_Dias_Ausente,Promedio_Horas_Extra
0,E001,Empleado_1,Recursos Humanos,Analista,1757,2021-02-07,F,3,0.333333,8.000000
1,E002,Empleado_2,Ventas,Director,2734,2021-04-18,F,3,0.666667,4.000000
2,E003,Empleado_3,Finanzas,Especialista,2432,2020-02-29,M,4,0.666667,8.333333
3,E004,Empleado_4,Ventas,Especialista,2806,2023-06-03,F,3,0.333333,8.333333
4,E005,Empleado_5,Ventas,Gerente,3349,2020-05-03,M,2,0.333333,9.666667


### Paso 2: Análisis y Generación de Información (KPIs)
Responde las preguntas del Director filtrando y ordenando los datos:

**A. Identificación de Talentos (Los "Estrellas")**
Filtra la lista para encontrar empleados que cumplan AMBAS condiciones:
1.  `Puntaje_Desempeno` sea igual a **4 o 5**.
2.  `Promedio_Dias_Ausente` sea **menor a 1**.
* *Pregunta 1:* ¿Cuántos empleados cumplen esto? ¿Quiénes son?

**B. Auditoría de "Sueldo vs. Rendimiento" (La Incoherencia)**
Queremos ver quiénes ganan mucho pero rinden poco.
1.  Ordena la tabla por `Salario_Mensual` de mayor a menor.
2.  Filtra aquellos que tengan un `Puntaje_Desempeno` de **1 o 2**.
* *Pregunta 2:* ¿Existe algún empleado con salario alto (ej. > $2000) que tenga bajo desempeño?

**C. Análisis Departamental (Carga de Trabajo)**
1.  Agrupa los datos por `Departamento`.
2.  Calcula el promedio de `Horas_Extra` por departamento.
* *Pregunta 3:* ¿Qué departamento está haciendo más horas extra? ¿Deberíamos contratar más personal allí?

# Talentos Estrella
En la siguiente tabla tenemos los empleados mas destacados, incluyendo el numero total de ellos.

In [91]:
#Identificación de Talentos ("Estrellas")
estrellas = df[(df["Puntaje_Desempeno"] >= 4) & (df["Promedio_Dias_Ausente"] < 1)]

print("El numero de empleados estrella es:", len(estrellas))
estrellas[["ID_Empleado","Nombre","Departamento","Cargo",
                   "Puntaje_Desempeno","Promedio_Dias_Ausente"]]


El numero de empleados estrella es: 20


,ID_Empleado,Nombre,Departamento,Cargo,Puntaje_Desempeno,Promedio_Dias_Ausente
2,E003,Empleado_3,Finanzas,Especialista,4,0.666667
11,E012,Empleado_12,Operaciones,Analista,4,0.333333
15,E016,Empleado_16,Operaciones,Especialista,5,0.000000
16,E017,Empleado_17,Finanzas,Especialista,4,0.000000
18,E019,Empleado_19,Finanzas,Gerente,4,0.000000
19,E020,Empleado_20,Ventas,Gerente,4,0.666667
20,E021,Empleado_21,TI,Director,4,0.000000
23,E024,Empleado_24,Ventas,Especialista,4,0.000000
24,E025,Empleado_25,Ventas,Especialista,4,0.000000
26,E027,Empleado_27,Finanzas,Director,4,0.333333


#Empleados que difieren en Sueldo vs Rendimiento
La siguiente tabla muestra los empleados que tienen el menor desempeño, mientras su sueldo excede los $2000.

In [92]:
#Auditoría de "Sueldo vs. Rendimiento"
incoherentes = df[(df["Puntaje_Desempeno"] <= 2) & (df["Salario_Mensual"] > 2000)]
incoherentes = incoherentes.sort_values(by="Salario_Mensual", ascending=False)

print("Empleados con salario alto y bajo desempeño:")
incoherentes[["ID_Empleado","Nombre","Departamento","Cargo",
                      "Salario_Mensual","Puntaje_Desempeno"]]

Empleados con salario alto y bajo desempeño:


,ID_Empleado,Nombre,Departamento,Cargo,Salario_Mensual,Puntaje_Desempeno
10,E011,Empleado_11,Ventas,Analista,3464,2
4,E005,Empleado_5,Ventas,Gerente,3349,2
34,E035,Empleado_35,TI,Especialista,2903,2
12,E013,Empleado_13,Operaciones,Gerente,2519,1


#Promedio de horas de los diferentes departamentos
Podemos observar cuales son los promedios de horas de las diferentes areas ordenadas de mayor a menor.

In [93]:
# Análisis Departamental (Carga de Trabajo)
carga_trabajo = df.groupby("Departamento")["Promedio_Horas_Extra"].mean().reset_index()
carga_trabajo = carga_trabajo.sort_values(by="Promedio_Horas_Extra", ascending=False)

print("Promedio de horas extra por departamento:")
carga_trabajo

Promedio de horas extra por departamento:


,Departamento,Promedio_Horas_Extra
4,Ventas,5.461538
2,Recursos Humanos,5.333333
3,TI,5.233333
1,Operaciones,5.166667
0,Finanzas,5.000000
